## User parameters

In [ ]:
path='/path/to/data'
filename='*.fepout'

temperature = 303.15 #in Kelvin

decorrelate = True #Flag for decorrelation of samples
detectEQ = True #Flag for automated equilibrium detection

## Imports and constants

In [ ]:
import safep
from safep import convergence_plot, plot_general
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

from scipy.constants import R
from scipy.constants import calorie
kcal = calorie*1000

#Suppress Pandas future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

RT = (R/kcal) * temperature
fepoutFiles = glob(path+filename)

## Read and process

In [ ]:
u_nk = safep.read_and_process(fepoutFiles, temperature, decorrelate, detectEQ)

## Do the BAR fitting and get ΔG
**Note: alchemlyb operates in units of kT by default. We multiply by RT to convert to units of kcal/mol.**

In [ ]:
perWindow, cumulative = safep.do_estimation(u_nk) #Run the BAR estimator on the fep data
forward, forward_error, backward, backward_error = safep.do_convergence(u_nk) #Used later in the convergence plot'
per_lambda_convergence = safep.do_per_lambda_convergence(u_nk)

In [ ]:
dG = np.round(cumulative.BAR.f.iloc[-1]*RT, 1)
error = np.round(cumulative.BAR.errors.iloc[-1]*RT, 1)

changeAndError = f'\u0394G = {dG}\u00B1{error} kcal/mol'
Markdown('<font size=5>{}</font><br/>'.format(changeAndError))

## Generate Plots 
- General plots:
Cumulative and per-window delta G.
- Convergence plots: 
Plot the estimated total change in free energy as a function of simulation time; contiguous subsets starting at t=0 ("Forward") and t=end ("Reverse")
- Per-window Convergence:
Difference between BAR estimates using first and last halves of the samples (with respect to simulation time)

In [ ]:
fig, axes = plot_general(cumulative, None, perWindow, None, RT)
fig.suptitle(changeAndError)
plt.savefig(f'{path}FEP_general_figures.pdf')

In [ ]:
fig, convAx = plt.subplots(1,1)
convAx = convergence_plot(convAx, forward*RT, forward_error*RT, backward*RT, backward_error*RT)
plt.savefig(f'{path}FEP_convergence.pdf')

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(per_lambda_convergence.index, per_lambda_convergence.BAR.df*RT)
ax.set_xlabel(r"$\lambda$")
ax.set_ylabel(r"$D_{last-first}$ (kcal/mol)")
plt.savefig(f"{path}FEP_perLambda_convergence.pdf")